# Intro

Pandas has first-class support for datetime types, including flexible indexing, vectorized operations, `groupby` operations and joins. This makes EDA on time series data with Pandas very convenient and productive.

In [1]:
%pylab inline
plt.style.use('bmh')

import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

Populating the interactive namespace from numpy and matplotlib


C:\Users\jjlev\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.read_csv('archive.zip')

In [2]:
DATA_DIR = pathlib.Path("archive/")

# Loading data

The dataset we'll use to explore time series functionality in Pandas is [1.6 million UK traffic accidents](https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales). The full dataset contains years 2005-2007, but note that in the Exam we only use 2005.

In [4]:
d = pd.read_csv(DATA_DIR.joinpath('accidents_2005_to_2007.csv'))

C:\Users\jjlev\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dataset is quite large. Let's explore it's per-column breakdown:

In [16]:
d.Date.dtypes == np.float64

False

In [18]:
d['new_date']=pd.to_datetime(d.Date)

In [ ]:
d['Accident_Severity< 3']

In [31]:
a_list =[]
top_5= d.query('Accident_Severity< 3').new_date.value_counts().head(5).index
for i in top_5:
    a_list.append(pd.Timestamp(i))
a_list
    

[Timestamp('2006-10-06 00:00:00'),
 Timestamp('2006-09-09 00:00:00'),
 Timestamp('2005-05-14 00:00:00'),
 Timestamp('2005-06-18 00:00:00'),
 Timestamp('2005-09-16 00:00:00')]

In [19]:
d.dtypes

Accident_Index                                         object
Location_Easting_OSGR                                 float64
Location_Northing_OSGR                                float64
Longitude                                             float64
Latitude                                              float64
Police_Force                                            int64
Accident_Severity                                       int64
Number_of_Vehicles                                      int64
Number_of_Casualties                                    int64
Date                                                   object
Day_of_Week                                             int64
Time                                                   object
Local_Authority_(District)                              int64
Local_Authority_(Highway)                              object
1st_Road_Class                                          int64
1st_Road_Number                                         int64
Road_Typ

In [5]:
d.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570011 entries, 0 to 570010
Data columns (total 33 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Accident_Index                               570011 non-null  object 
 1   Location_Easting_OSGR                        569910 non-null  float64
 2   Location_Northing_OSGR                       569910 non-null  float64
 3   Longitude                                    569910 non-null  float64
 4   Latitude                                     569910 non-null  float64
 5   Police_Force                                 570011 non-null  int64  
 6   Accident_Severity                            570011 non-null  int64  
 7   Number_of_Vehicles                           570011 non-null  int64  
 8   Number_of_Casualties                         570011 non-null  int64  
 9   Date                                         570011 non-nul

As we can see, it has data and time separately, and we need to combine it into full datetime:

In [6]:
d.Date.head()

0    04/01/2005
1    05/01/2005
2    06/01/2005
3    07/01/2005
4    10/01/2005
Name: Date, dtype: object

In [7]:
d.Time.head()

0    17:42
1    17:36
2    00:15
3    10:35
4    21:13
Name: Time, dtype: object

Let's explore if we have any missing dates or times:

In [8]:
d[d.Date.isnull()]

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year


In [9]:
d[d.Time.isnull()]

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
29084,200504AL05142,342200.0,432250.0,-2.878714,53.783349,4,3,2,1,30/11/2005,...,NaN,Daylight: Street light present,Unknown,NaN,NaN,NaN,1,NaN,E01025009,2005
110943,2.01E+12,554450.0,281270.0,0.269172,52.407602,35,3,1,1,09/02/2005,...,No physical crossing within 50 meters,Darkness: Street lighting unknown,Unknown,Dry,None,None,1,No,E01018028,2005
110990,2.01E+12,542970.0,259050.0,0.091097,52.211087,35,3,2,1,16/02/2005,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,No,E01017984,2005
111018,2.01E+12,517770.0,281930.0,-0.269528,52.422651,35,3,1,1,20/02/2005,...,No physical crossing within 50 meters,Darkness: Street lighting unknown,Unknown,Dry,None,None,2,Yes,E01018192,2005
111607,2.01E+12,516440.0,302060.0,-0.282036,52.603825,35,3,2,1,06/05/2005,...,No physical crossing within 50 meters,Darkness: Street lighting unknown,Unknown,Dry,None,None,1,No,E01015593,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563752,2.01E+12,280300.0,686760.0,-3.923863,56.058715,96,3,1,1,06/05/2007,...,No physical crossing within 50 meters,Darkeness: No street lighting,Fine without high winds,Wet/Damp,None,None,2,Yes,NaN,2007
563813,2.01E+12,299650.0,680980.0,-3.611104,56.011255,96,3,2,1,09/06/2007,...,Pedestrian phase at traffic signal junction,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,NaN,2007
563952,2.01E+12,288240.0,682680.0,-3.794724,56.023996,96,2,2,1,23/08/2007,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,NaN,2007
563970,2.01E+12,280630.0,680670.0,-3.915856,56.004111,96,2,1,1,02/09/2007,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,2,Yes,NaN,2007


We may note the following:
    
- date and time are provided as strings,
- we have slashes in dates, and this can be parsed ambiguously,
- some times are missing.

Hence, our strategy is the following:

- concatenate date and time using string vectorized operations,
- set placeholder for missing times to be `00:00`,
- parse resulting (**string**) datetime with explicit `dayfirst=True`.

In [32]:
d.loc[:, 'dt'] = d.Date.str.cat(d.Time, sep=' ', na_rep='00:00')

In [14]:
d.Date.dtype

dtype('O')

In [34]:
d.loc[:, 'date_time'] = pd.to_datetime(d.dt, dayfirst=True)

We now have `date_time` column of type `datetime64[ns]`:

In [37]:
d.date_time.dt.date.value_counts()  # Note the difference without `memory_usage="deep"`

2005-10-21    822
2005-11-18    787
2006-09-29    784
2006-09-22    780
2005-12-07    775
             ... 
2007-12-31    249
2005-12-25    199
2007-12-30    192
2006-12-25    175
2007-12-25    157
Name: date_time, Length: 1095, dtype: int64

In [38]:
def worst_days(df):
    a_list =[]
    top_5= df.query('Accident_Severity< 3').date_time.dt.date.value_counts().head(5).index
    for i in top_5:
        a_list.append(pd.Timestamp(i))
    return a_list


In [39]:
worst_days(d)

[Timestamp('2006-06-10 00:00:00'),
 Timestamp('2006-09-09 00:00:00'),
 Timestamp('2005-05-14 00:00:00'),
 Timestamp('2005-06-18 00:00:00'),
 Timestamp('2005-09-16 00:00:00')]

Let's filter some columns we do not need:

In [ ]:
d.columns

In [ ]:
COLS = ['Accident_Index', 'Longitude', 'Latitude',
        'Accident_Severity', 'Number_of_Vehicles',
        'Number_of_Casualties', 'Weather_Conditions',
        'Day_of_Week', 'Road_Surface_Conditions',
        'Special_Conditions_at_Site', 'Urban_or_Rural_Area',
        'Carriageway_Hazards', 'date_time']

In [ ]:
d.drop([c for c in d.columns if c not in COLS], axis=1, inplace=True)

Pandas has a dedicated set of index types for datetime indexes:

In [ ]:
d.set_index('date_time', inplace=True)

In [ ]:
d.index

In [ ]:
d.head()

# `DatetimeIndex` in details

`DatetimeIndex` is special in many ways. It allows for much more flexible indexing compared to usual indexes. First of all, you can use strings, not just actual index labels. To leverage this, we first sort the index:

In [ ]:
d.sort_index(inplace=True)

We can now use strings to index the dataframe (indexing non-monotonic `DatetimeIndex` with strings is not a very good idea):

In [ ]:
d["2006-02-12 20":"2006-03"]

In [ ]:
d["2005":]

Note, how Pandas allows for partial datetime string specification. Of course, this way of indexing can be combined with column index:

In [ ]:
d.loc["2005", "Accident_Severity"]

We will now create a dataframe used in Problem 6 of the exam:

In [ ]:
d.loc["2005", "Accident_Severity"].to_csv(DATA_DIR.joinpath("accidents_2005.csv"))

In [ ]:
accidents_2005 = pd.read_csv(DATA_DIR.joinpath("accidents_2005.csv"),
                             parse_dates=["date_time"])

In [ ]:
accidents_2005.head()

Note, that it's not indexed, and that's exactly the way it's passed to the solution function.

# Resampling time series

Time series in Pandas can be easily resampled to any frequency:

In [ ]:
d.resample('D')

Similar to `groupby`, `resample` doesn't perform any operations on it's own, but just calculates which rows go to which (datetime) bin. We need to further apply some aggregation operation. For example, we may calculate number of accidents per day:

In [ ]:
daily = d.resample('D').size()
daily

`daily` has `DatetimeIndex` as well and has `freq` specified (as it was constructed to have one):

In [ ]:
daily.index

In [ ]:
daily.index.is_monotonic, daily.index.is_unique

Pandas also exposes plotting functionality to datetime-indexed dataframes. To illustrate this, let's plot daily and weekly average number of accidents:

In [ ]:
# Just a hint: you can set image resolution in dpi
plt.figure(figsize=(8,3), dpi=150)  

daily.plot(ax=plt.gca(), linewidth=0.5)

(d.resample('W').size()/7.).plot(ax=plt.gca(),
                                 linewidth=1,
                                 color='firebrick')

plt.ylabel('accidents')
plt.xlabel('month');

In EDA terms, we just gained our first insight: accidents are strongly seasonal (with non-trivial seasonal struture and high dependence on holidays).

Similarly, we can plot daily and weekly average number of vehicles involved:

In [ ]:
plt.figure(figsize=(12,5), dpi=150)

d.resample('D').Number_of_Vehicles.mean().plot(ax=plt.gca())
d.resample('W').Number_of_Vehicles.mean().plot(ax=plt.gca(), color='firebrick')

plt.ylabel('vehicles involved')
plt.xlabel('month');

Instead of `resample`, we can use `pd.Grouper`. It's not really useful as a replacement of `resample`, but is very handy in compound grouping keys.

In [ ]:
d.groupby(pd.Grouper(freq='D'))["Number_of_Casualties"].mean()

Now, let's try to extract the accidents, which have more casualties, than average number of casualties on that day. And now Pandas datetime magic comes into play:

In [ ]:
daily_casualties = (d
                    .groupby(pd.Grouper(freq='D'))["Number_of_Casualties"]
                    .mean())

df = d.merge(daily_casualties,
             left_on=d.index.floor("1D"),
             right_index=True,
             suffixes=("", "_daily"))

Note that Pandas keeps the calculated key it used for merging as `key_0`:

In [ ]:
df.head()

We do not need it at the moment, so we'll drop it:

In [ ]:
df.drop("key_0", axis=1, inplace=True)

We can now calculate how extreme each accident is compared to daily averages:

In [ ]:
df["delta"] = df["Number_of_Casualties"] - df["Number_of_Casualties_daily"]

In [ ]:
df.loc["2005"].sort_values(by="delta", ascending=False)

Let's explore the most extreme one:

In [ ]:
d[d.Accident_Index=="200597EC70504"]

You may want to further investigate this case (with Google of course).

Let's get back to `pd.Grouper` and compound keys. We can flexibly combine grouper on datetime index with usual column. Let's calculate now many accidents we have per area type each day:

In [ ]:
d.groupby([pd.Grouper(freq='1M'), 'Urban_or_Rural_Area']).size()

Now we can plot this as a stacked bar plot:

In [ ]:
plt.figure(figsize=(12,6))

(d.groupby([pd.Grouper(freq='1M'), 'Urban_or_Rural_Area'])
 .size()
 .unstack()
 .plot(alpha=0.6, linewidth=2, ax=plt.gca(), kind='bar', stacked=True));

We can do the same with accident severity:

In [ ]:
plt.figure(figsize=(12,6))

(d.groupby([pd.Grouper(freq='1M'), 'Accident_Severity'])
 .size()
 .unstack()
 .plot(alpha=0.6, linewidth=2, ax=plt.gca(), kind='bar', stacked=True));